# Spot the difference

So in this num(meth)math project I shall be doing a basic
spotting the difference program \
\
Theres two method of doing this, one method is the easy way of importing a library and everything could be done in 4 lines of code and of course the hard way which is doing it using matplotlib, and we will doing both method, because where is the fun in just importing a library that could do that in two lines

## Image Spliter
\
If the image to compare is one picture we have to split the image using a image-slicer library \
We have to install the image_slicer library using \
``pip install image_slicer``

In [93]:
# Import the good stuff

from image_slicer import slice

In [95]:
# Path of the image

path = "Split/image.jpeg"

# Slice the image into 2

slice(path, 2)

(<Tile #1 - image_01_01.png>, <Tile #2 - image_01_02.png>)

### Now we have two images

<table><tr>
<td> <img src="Split/image_01_01.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="Split/image_01_02.png" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>